# Creating XYZ-format files with ASE
With the ase package open the cif file and create a supercell with the .repeat() function. The repeated structure is saved to a .xyz file that is more suitable for MotEx.

In [1]:
from ase import io
import ipywidgets 
from ase.visualize import view

In [2]:
stru = io.read('Structure_Models/beta-Ga2O3_CollCode83645.cif')

In [3]:
print(stru.cell)
print(len(stru.get_positions()))

Cell([[12.214, 0.0, 0.0], [0.0, 3.0371, 0.0], [-1.3859888959218767, 0.0, 5.630008738037735]])
20


In [4]:
supercell = stru.repeat((2,2,2))

In [5]:
len(supercell.get_positions())

160

In [6]:
view(supercell, viewer='x3d')

In [7]:
supercell.write('Structure_Models/GO_2x2x2.xyz')

In [18]:
ga = list(supercell.get_atomic_numbers()).count(31)
o = list(supercell.get_atomic_numbers()).count(8)
print(f'There are {ga} Ga and {o} O atoms in the supercell.')

There are 64 Ga and 96 O atoms in the supercell.
